### Complejidad - Práctico 5/6 - Juegos evolutivos
#### Lautaro Giordano

In [1]:
import numpy as np
from matplotlib import pyplot as plt

In [284]:
class Equipo():
    def __init__(self, ataque=None, presion=None):
        # Variables que caracterizan al equipo
        self.ataque = ataque if ataque is not None else np.random.rand()
        self.defensa = 1 - self.ataque
        self.presion = presion if presion is not None else np.random.rand()
        self.energia = 100
        # El nombre del equipo codifica el ataque, defensa y presion de esta forma "AADDPP" donde AA son las primeras dos cifras de 100*ataque
        # El nombre "901040" significa que el equipo tiene un ataque del 90%, una defensa del 10% y una presion del 40%
        self.name = "{:02d}|{:02d}|{:02d}".format(round(100*self.ataque), round(100*self.defensa), round(100*self.presion))
        
        # Variables de los partidos jugados
        self.puntos = 0
        self.partidos_ganados = 0
        self.partidos_empatados = 0
        self.partidos_perdidos = 0
        self.goles_favor = 0
        self.goles_contra = 0

        # Variables de la evolución
        self.fitness = 0


class Liga():
    def __init__(self, num_equipos, descanso=50):
        self.equipos = [Equipo() for _ in range(num_equipos)]
        self.num_equipos = num_equipos
        self.descanso = descanso
        self.calendario = self.calendario_liga()
        self.goles_totales = 0
        self.cant_partidos = 0
        self.promedio_goles = 0

    def reset_energia(self):
        for equipo in self.equipos:
            equipo.energia = 100

    def partido(self, equipo1, equipo2):
        resultado = 0
        for min in range(90):
            if np.random.rand() < equipo1.ataque:
                if np.random.rand() < equipo2.defensa:
                    equipo1.goles_favor += 1
                    equipo2.goles_contra += 1
                    resultado += 1
            elif np.random.rand() < equipo2.ataque:
                if np.random.rand() < equipo1.defensa:
                    equipo2.goles_favor += 1
                    equipo1.goles_contra += 1
                    resultado -= 1

            equipo1.energia -= equipo1.presion
            equipo2.energia -= equipo2.presion
        
        if resultado > 0:
            equipo1.puntos += 3
            equipo1.partidos_ganados += 1
            equipo2.partidos_perdidos += 1
        elif resultado < 0:
            equipo2.puntos += 3
            equipo2.partidos_ganados += 1
            equipo1.partidos_perdidos += 1
        else:
            equipo1.puntos += 1
            equipo2.puntos += 1
            equipo1.partidos_empatados += 1
            equipo2.partidos_empatados += 1

        # Simulo descanso
        equipo1.energia += 60
        equipo2.energia += 60

    def partido(self, equipo1, equipo2):
        resultado = 0
        for minuto in range(90):
            fuerza1 = equipo1.energia * equipo1.presion
            fuerza2 = equipo2.energia * equipo2.presion
            # Probabilidad de que la ocasión de gol sea del equipo1 en base a la presión y energia de los equipos
            p1 = fuerza1 / (fuerza1 + fuerza2) if fuerza1 + fuerza2 > 0 else 0.5

            if (equipo1.energia <= 0 and equipo2.energia <= 0):
                # Si ninguno de los dos equipos tiene energía, terminamos el partido
                break

            # Simulamos si ocurre una ocasión de gol
            if np.random.rand() < 0.15:  # Los partidos tipicos de la liga tienen unos 20 tiros al arco.
                if np.random.rand() < p1:
                    # Ocasión de gol para equipo1
                    if np.random.rand() < equipo1.ataque and np.random.rand() > equipo2.defensa:
                        equipo1.goles_favor += 1
                        equipo2.goles_contra += 1
                        resultado += 1
                else:
                    # Ocasión de gol para equipo2
                    if np.random.rand() < equipo2.ataque and np.random.rand() > equipo1.defensa:
                        equipo2.goles_favor += 1
                        equipo1.goles_contra += 1
                        resultado -= 1

            # Actualizar energía de los equipos (aca juega un rol la presion, que esta rescaleada para que si presion=1, un partido entero resta 100 de energia)
            equipo1.energia = max(0, equipo1.energia - (100/90) * equipo1.presion)
            equipo2.energia = max(0, equipo2.energia - (100/90) * equipo2.presion)
    
        # Actualizar puntos y resultados
        if resultado > 0:
            equipo1.puntos += 3
            equipo1.partidos_ganados += 1
            equipo2.partidos_perdidos += 1
        elif resultado < 0:
            equipo2.puntos += 3
            equipo2.partidos_ganados += 1
            equipo1.partidos_perdidos += 1
        else:
            equipo1.puntos += 1
            equipo2.puntos += 1
            equipo1.partidos_empatados += 1
            equipo2.partidos_empatados += 1

        # Simulo descanso
        equipo1.energia = min(100, equipo1.energia + self.descanso)  # Recuperar hasta un máximo de 100 de energía
        equipo2.energia = min(100, equipo2.energia + self.descanso)
   
    def calendario_liga(self):
        equipos = list(range(self.num_equipos))
        
        # Si el número de equipos es impar, añadimos un "bye"
        if len(equipos) % 2 != 0:
            equipos.append(None)  # "None" representa un equipo libre
        
        num_equipos = len(equipos)
        calendario = []

        for fecha in range(num_equipos - 1):
            jornada = []
            for i in range(num_equipos // 2):
                equipo1 = equipos[i]
                equipo2 = equipos[num_equipos - 1 - i]
                
                # Solo agregamos el partido si ninguno de los dos es "None"
                if equipo1 is not None and equipo2 is not None:
                    jornada.append((equipo1, equipo2))
            
            calendario.append(jornada)
            
            # Rotamos los equipos (sin incluir el primer equipo)
            equipos = [equipos[0]] + equipos[-1:] + equipos[1:-1]

        return calendario
    
    def simular_liga(self):
        for jornada in self.calendario:
            for partido in jornada:
                if None not in partido:
                    self.partido(self.equipos[partido[0]], self.equipos[partido[1]])
                    
        self.reset_energia()

        self.goles_totales = sum([equipo.goles_favor for equipo in self.equipos])
        self.cant_partidos += (self.num_equipos - 1) * self.num_equipos // 2
        self.promedio_goles = self.goles_totales / self.cant_partidos

    def imprimir_tabla(self, orden='puntos'):
        print("Nombre\t\tPuntos\tPG\tPE\tPP\tGF\tGC\tDif")
        if orden == 'puntos':
            equipos_ordenados = sorted(self.equipos, key=lambda x: (x.puntos, x.goles_favor - x.goles_contra), reverse=True)
        elif orden == 'goles':
            equipos_ordenados = sorted(self.equipos, key=lambda x: (x.goles_favor - x.goles_contra, x.puntos), reverse=True)

        for equipo in equipos_ordenados:
            print(f"{equipo.name}\t{equipo.puntos}\t{equipo.partidos_ganados}\t{equipo.partidos_empatados}\t{equipo.partidos_perdidos}\t{equipo.goles_favor}\t{equipo.goles_contra}\t{equipo.goles_favor - equipo.goles_contra}")
        
        print(f"Goles totales: {self.goles_totales}, Goles por partido: {self.promedio_goles:.2f}")


In [287]:
liga = Liga(num_equipos=500, descanso=50)
# Imprimo el calendario
for _ in range(1):
    liga.simular_liga()
    
liga.imprimir_tabla()

Nombre		Puntos	PG	PE	PP	GF	GC	Dif
89|11|50	1217	382	71	46	2318	784	1534
88|12|49	1207	378	73	48	2269	778	1491
89|11|41	1205	373	86	40	2177	814	1363
84|16|50	1200	378	66	55	2095	727	1368
93|07|47	1197	377	66	56	2386	865	1521
90|10|46	1190	373	71	55	2244	844	1400
93|07|45	1184	376	56	67	2361	853	1508
94|06|42	1170	366	72	61	2338	899	1439
90|10|43	1165	364	73	62	2220	875	1345
73|27|48	1163	360	83	56	1848	645	1203
89|11|44	1161	362	75	62	2190	818	1372
99|01|45	1155	365	60	74	2506	903	1603
81|19|36	1142	356	74	69	1972	780	1192
62|38|47	1142	349	95	55	1596	553	1043
76|24|43	1141	354	79	66	1857	718	1139
72|28|45	1139	351	86	62	1793	671	1122
68|32|46	1138	349	91	59	1704	588	1116
80|20|37	1134	348	90	61	1988	873	1115
71|29|46	1134	348	90	61	1700	666	1034
70|30|46	1131	348	87	64	1764	627	1137
87|13|37	1127	347	86	66	2112	860	1252
53|47|50	1121	337	110	52	1362	432	930
63|37|48	1117	342	91	66	1518	549	969
90|10|39	1115	340	95	64	2125	892	1233
66|34|38	1110	340	90	69	1585	616	969
52|48|48	1109	328	